In [3]:
import numpy as np
import pandas as pd
import random

import os
import pickle

import sys
sys.path.insert(0, '../')

import constants as constants

from features.mimic3_function import *
from visualization.patientlevel_function  import *
from visualization.plot_functions import *
from visualization.table_functions import *
from constants import *

import visualization.plot_functions as fprs_tprs_output

### Getting Train set results (auc and confidence interval)

In [50]:

if __name__ == '__main__':
    print("Collecting results at setting x,y,T=24,12,6 from three models", MODELS[0],MODELS[1],MODELS[-1])
    
    current_data=exclusion_rules[0]
    
    Root_Data=DATA_processed+current_data
    Data_Dir=Root_Data+'/train/'
    print("The interim results will be collected from ", Data_Dir)
    
    x,y,a1=24,12,6
    train_labels_list_list=[]
    train_probs_list_list=[]
    train_indices_list_list=[]
    train_icuid_sequence_list_list=[]
    n=100
    for model in MODELS:
        
        print("Now Collecting results from model", model)
        
        labels_list=[]
        probs_list=[]
        test_indices_list=[]

        _, _, Output_predictions, _ = folders(current_data, model=model)
        for definition in FEATURES:
            

            labels_now = np.load(Data_Dir + 'label_' + str(x) +
                                 '_' + str(y) + '_' + str(a1) + definition[1:] + '.npy')
            
            icu_lengths_now = np.load(
                Data_Dir + 'icustay_lengths_' + str(x) + '_' + str(y) + definition[1:] + '.npy')
            

            probs_now=np.load(
                Output_predictions + '/train/' + str(x) + '_' + str(y) + '_' + str(a1) + definition[1:] + '.npy')
            
            print(len(labels_now),len(probs_now))
            
            icustay_fullindices_now=patient_idx(icu_lengths_now)
                        
            labels_list.append(labels_now)
            probs_list.append(probs_now)
            test_indices_list.append(icustay_fullindices_now)


        train_labels_list_list.append(labels_list)
        train_probs_list_list.append(probs_list)
        train_indices_list_list.append(test_indices_list)
    
    ######### Instance level auc plot/table #################
    print("Instance level now:")

The interim results will be collected from  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/data/processed/blood_only_hos_stay/train/
Now Collecting results from model LSTM
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/models/blood_only_hos_stay/LSTM/  already exists
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/outputs/predictions/blood_only_hos_stay/LSTM/  already exists
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/outputs/results/blood_only_hos_stay/LSTM/  already exists
355742 355742
366450 366450
352349 352349
Now Collecting results from model CoxPHM
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/models/blood_only_hos_stay/CoxPHM/  already exists
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/outputs/predictions/blood_only_hos_stay/CoxPHM/  already exists
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/outputs/results/blood_only_hos_stay/CoxPHM/  already exists
355742 355742
366450 366450
352349 352349
Instance level now

In [52]:
fprs_train_lists, tprs_train_lists=fprs_tprs_output(train_labels_list_list,train_probs_list_list,MODELS=MODELS)

LSTM t_sofa 0.830 [0.826,0.834]
LSTM t_suspicion 0.871 [0.868,0.873]
LSTM t_sepsis_min 0.803 [0.798,0.808]


CoxPHM t_sofa 0.796 [0.792,0.801]
CoxPHM t_suspicion 0.850 [0.846,0.853]
CoxPHM t_sepsis_min 0.785 [0.780,0.790]




In [55]:
for i in range(len(MODELS)):
    
    for j in range(len(FEATURES)):

        train_full_idxs=train_indices_list_list[i][j]
        par_probs=[patient_level_probability_max(train_probs_list_list[i][j][train_full_idxs[k]]) for k in range(len(train_full_idxs))]

        par_labels=[train_labels_list_list[i][j][train_full_idxs[k]][-1] for k in range(len(train_full_idxs))]
        
        
        CI_results,fprs,tprs=CI_AUC_bootstrapping(n_bootstraps, alpha, np.array(par_labels), np.array(par_probs),  rng_seed = 1)
        
        print(MODELS[i],FEATURES[j],"{:.3f}".format(roc_auc_score(par_labels,par_probs)),"["+"{:.3f}".format(CI_results[0]) +","+"{:.3f}".format(CI_results[1])+"]")
        print()
    print('\n')      

LSTM t_sofa 0.740 [0.727,0.755]

LSTM t_suspicion 0.805 [0.793,0.817]

LSTM t_sepsis_min 0.770 [0.749,0.788]



CoxPHM t_sofa 0.691 [0.677,0.706]

CoxPHM t_suspicion 0.792 [0.781,0.804]

CoxPHM t_sepsis_min 0.722 [0.699,0.742]





### Forming performance table

In [36]:
def getting_written_table(test_result_dfs):
    outputs=[]

    for j in range(len(constants.xy_pairs)):
    
        for a1 in constants.T_list:
        
            (x,y)=xy_pairs[j]

            df_now=test_result_dfs[(test_result_dfs['x,y']==str(x)+','+str(y))&(test_result_dfs['T']==a1)]

            output=["{:.3f}".format(list(df_now.auc)[i])+'/'+"{:.3f}".format(list(df_now.speciticity)[i])+'/'+\
            "{:.3f}".format(list(df_now.accuracy)[i])+' &' for i in range(len(constants.FEATURES))]

            outputs.append([str(x)+','+str(y),a1,output[0],output[1],output[-1]])
    test_output_df= pd.DataFrame(outputs, columns=['x,y', 'T','H1','H2','H3'])
    
    return test_output_df

### a new one to add sensitivity too

In [ ]:
def getting_written_table_test(test_result_dfs):
    outputs=[]

    for j in range(len(constants.xy_pairs)):
    
        for a1 in constants.T_list:
        
            (x,y)=xy_pairs[j]

            df_now=test_result_dfs[(test_result_dfs['x,y']==str(x)+','+str(y))&(test_result_dfs['T']==a1)]

            output=["{:.3f}".format(list(df_now.auc)[i])+'/'+"{:.3f}".format(list(df_now.sensitivity)[i])+'/'+"{:.3f}".format(list(df_now.speciticity)[i])+'/'+\
            "{:.3f}".format(list(df_now.accuracy)[i])+' &' for i in range(len(constants.FEATURES))]

            outputs.append([str(x)+','+str(y),a1,output[0],output[1],output[-1]])
    test_output_df= pd.DataFrame(outputs, columns=['x,y', 'T','H1','H2','H3'])
    
    return test_output_df

In [33]:
current_data=exclusion_rules[0]

for model in constants.MODELS[:1]:
    _, _, Output_predictions, _ = folders(current_data, model=model)
    train_result_dfs=pd.read_csv(Output_predictions+'train/lgbm_train_results.csv')

Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/models/blood_only_hos_stay/LGBM/  already exists
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/outputs/predictions/blood_only_hos_stay/LGBM/  already exists
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/outputs/results/blood_only_hos_stay/LGBM/  already exists


In [37]:
getting_written_table(train_result_dfs)

,"x,y",T,H1,H2,H3
0,"48,24",12,0.846/0.661/0.668 &,0.908/0.788/0.792 &,0.838/0.622/0.627 &
1,"48,24",8,0.849/0.665/0.670 &,0.911/0.797/0.799 &,0.846/0.644/0.648 &
2,"48,24",6,0.854/0.676/0.680 &,0.909/0.793/0.795 &,0.852/0.672/0.675 &
3,"48,24",4,0.857/0.686/0.689 &,0.908/0.794/0.795 &,0.857/0.676/0.678 &
4,"24,12",12,0.854/0.672/0.677 &,0.920/0.820/0.821 &,0.832/0.615/0.620 &
5,"24,12",8,0.861/0.678/0.682 &,0.924/0.834/0.835 &,0.843/0.628/0.632 &
6,"24,12",6,0.866/0.694/0.697 &,0.922/0.832/0.833 &,0.851/0.650/0.653 &
7,"24,12",4,0.870/0.702/0.704 &,0.923/0.828/0.829 &,0.858/0.659/0.661 &
8,"12,6",12,0.865/0.695/0.699 &,0.935/0.858/0.858 &,0.840/0.630/0.635 &
9,"12,6",8,0.873/0.704/0.707 &,0.939/0.871/0.871 &,0.854/0.652/0.655 &


In [16]:
current_data=exclusion_rules[0]

for model in constants.MODELS[:1]:
    _, _, Output_predictions, _ = folders(current_data, model=model)
    test_result_dfs=pd.read_csv(Output_predictions+'test/lgbm_test_results.csv')

Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/models/blood_only_hos_stay/LGBM/  already exists
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/outputs/predictions/blood_only_hos_stay/LGBM/  already exists
Directory  /scratch/wuy/Mimiciii_Sepsis_Label_Extraction/outputs/results/blood_only_hos_stay/LGBM/  already exists


In [ ]:
getting_written_table_test(test_result_dfs)